# Easy Ab initio calculation with ASE-Siesta-Pyscf

## No installation necessary, just download a ready to go container for any system, or run it into the cloud

### We first import the necessary libraries and define the system using ASE

In [ ]:
# import libraries and set up the molecule geometry

from ase.units import Ry, eV, Ha
from ase.calculators.siesta import Siesta
from ase import Atoms
import numpy as np
import matplotlib.pyplot as plt

from ase.build import molecule

CH4 = molecule("CH4")

# visualization of the particle
from ase.visualize import view
view(CH4, viewer='x3d')

### We can then run the DFT calculation using Siesta

In [ ]:
# enter siesta input and run siesta
siesta = Siesta(
    mesh_cutoff=150 * Ry,
    basis_set='DZP',
    pseudo_qualifier='lda',
    energy_shift=(10 * 10**-3) * eV,
    fdf_arguments={
        'SCFMustConverge': False,
        'COOP.Write': True,
        'WriteDenchar': True,
        'PAO.BasisType': 'split',
        'DM.Tolerance': 1e-4,
        'DM.MixingWeight': 0.1,
        'MaxSCFIterations': 300,
        'DM.NumberPulay': 4,
        'XML.Write': True})

CH4.set_calculator(siesta)
e = CH4.get_potential_energy()

### Calculating the dos and the pdos with PySCF-NAO

In [ ]:
# compute dos and pdos using pyscf-nao

from pyscf.nao import scf

dft = scf(label="siesta")
omegas = np.arange(0.0, 10.0, 0.05) + 1.0j*0.01
omegas/= Ha
dos = dft.dos(omegas)
pdos = dft.pdos(omegas)

In [ ]:
# plot polarizability with matplotlib
%matplotlib inline

fig = plt.figure(1)
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax1.plot(omegas*Ha, dos)
print(pdos.shape)

for i in range(pdos.shape[0]):
    ax2.plot(omegas*Ha, pdos[i, :])

ax1.set_xlabel(r"$\omega$ (eV)")
ax2.set_xlabel(r"$\omega$ (eV)")

ax1.set_ylabel(r"dos (au)")
ax2.set_ylabel(r"pdos (au)")

ax1.set_title(r"dos")
ax2.set_title(r"pdos")

fig.tight_layout()